# Lab — Prompt Engineering: Giving AI the Right Instructions

**Pre-Requisites:**
- [Create a Hugging Face account and retrieve access token](https://huggingface.co/docs/hub/en/security-tokens)
- [Add the Hugging Face token to your Colab secrets](https://pyimagesearch.com/2025/04/04/configure-your-hugging-face-access-token-in-colab-environment/)

**Goal:**  
Develop practical skills to structure prompts so that LLMs:

- Produce clearer, more constrained, and safer outputs.  
- Follow specific roles, formats, and step-by-step reasoning.  
- Are less likely to hallucinate or drift off-task in security contexts.


Large Language Models (LLMs) like GPT or OPT don’t think — they follow your instructions.
- Vague prompts = vague results
- Clear, structured prompts = focused, high-quality results

You’re not just “asking questions” — you’re **programming with natural language**.

---

### What is Prompt Engineering?

Prompt engineering is the art of **designing the input** to a model in a way that:
- Guides its tone and style
- Improves relevance and depth
- Reduces hallucinations and randomness
- Mimics roles, formats, or logic

You can shape output dramatically by changing *just a few words* in your input!

---

**What you're practicing**:
- Writing both a *simple* prompt and an *engineered* version
- Understanding how prompt structure changes the model’s output
- Learning prompt patterns like **role-based prompts**, **output templates**, and **context embedding**

**Why this matters**:
- Every real-world AI app — from customer support to marketing — depends on good prompts
- You can improve output without changing the model, data, or code
- Prompting is the easiest and most powerful way to make LLMs useful for *your* goals

---
## 0. Setup

Install libraries if needed and load a small model (GPT-2 or a small chat model).


In [ ]:
# If in Colab, uncomment:
!pip install -q transformers torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

---
## 1. Load a Model

Here we use `gpt2`, but for better instruction following you may swap it with a small chat-tuned model if available.


In [ ]:
model_name = "facebook/opt-1.3b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)
model.eval()

print("Loaded:", model_name)

---
## 2. Helper Function

We use a simple generation helper with modest sampling.


In [ ]:
def generate(prompt, max_new_tokens=160, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

---
## 3. Naive vs Structured Prompt

1. Start with a **basic question** like:  
   _“Explain least privilege.”_

2. Then create a **prompt-engineered version** using at least one of these templates:
   - “Act as a [role]...”  
   - “Use the format:...”  
   - “Explain it like I’m a beginner…”  
   - “Here’s context, now answer: …”

3. Generate both completions and compare:
   - Which one is clearer?
   - Which one gives more useful advice?
   - Which one sounds more professional?


In [ ]:
naive_prompt = "Explain least privilege."
better_prompt = (
    "You are a cloud security instructor. "
    "Use short paragraphs, a concrete cloud example, and finish with a 3-item checklist."
    "Explain to postgraduate students in a cloud security program."
    "Here’s context, now answer: Explain the principle of least privilege "
)

for label, p in [("Naive prompt", naive_prompt), ("Better structured prompt", better_prompt)]:
    print("=" * 80)
    print(label)
    print("=" * 80)
    print(generate(p))
    print("\n\n")

### ✅ TODO 1 – Improve a Prompt Yourself

1. Choose a simple security topic (e.g., "S3 bucket misconfigurations").  
2. Write a naive prompt.  
3. Write a more structured, role-based version that specifies audience, format, and constraints.

Fill in the TODO cell and compare outputs.


In [ ]:
# TODO: Fill in your own prompts

naive = "TODO: write a very short naive prompt here"
structured = (
    "TODO: rewrite the naive prompt as a detailed instruction. For example, specify:\n"
    "- Role (e.g., cloud security engineer)\n"
    "- Audience (e.g., junior analysts)\n"
    "- Format (e.g., bullet points, checklist)\n"
    "- Constraints (e.g., avoid jargon, include one AWS-specific example)\n"
)

for label, p in [("Naive", naive), ("Structured", structured)]:
    print("=" * 80)
    print(label)
    print("=" * 80)
    print(generate(p))
    print("\n\n")

---
## 4. Asking for Step-by-Step Reasoning (Conceptual Only)

Even if the underlying model is not a “reasoning model”, adding instructions like:

- “Think step by step.”  
- “First list your assumptions, then propose actions.”  

often changes the style of the answer.

We will try this on an incident response scenario.


In [ ]:
prompt_plain = (
    "Describe how you would respond to a suspected credential leak in a cloud environment."
)

prompt_step_by_step = (
    "You are an experienced incident responder. Describe how you would respond to a suspected "
    "credential leak in a cloud environment. Think step by step. First list key assumptions, then "
    "outline the investigation steps, and finally propose containment and recovery actions."
)

for label, p in [("Plain prompt", prompt_plain), ("Step-by-step prompt", prompt_step_by_step)]:
    print("=" * 80)
    print(label)
    print("=" * 80)
    print(generate(p))
    print("\n\n")

### ✅ TODO 2 – Design a “Runbook-Style” Prompt

Create a prompt that asks the model to output a **runbook** for a specific security scenario, e.g.:

- "Suspicious login from unknown country."  
- "Public GitHub repo containing secrets."

Your prompt should specify:

- Role (e.g., SOC analyst)  
- Format (e.g., headings: Detection, Triage, Containment, Eradication, Recovery)  
- Level of detail (e.g., 3–5 bullet points per section)


In [ ]:
# TODO: Design your runbook-style prompt
runbook_prompt = (
    "TODO: write a detailed instruction asking the model to output an incident response runbook "
    "for a scenario of your choice. Specify role, sections, and bullet-point requirements."
)

print(generate(runbook_prompt))

---
## 5. Prompting for Safer Behavior

Prompts can also explicitly request **safe, compliant** behavior, for example:

- “Do not provide any steps that would violate laws or company policies.”  
- “If you are unsure, say you are unsure rather than guessing.”

We will compare answers with and without such constraints.


In [ ]:
unsafe_like_prompt = (
    "Suggest ways to test the security of a web application."
)

safer_prompt = (
    "You are a security consultant. Suggest ways to test the security of a web application. "
    "Only include methods that are legal and ethical when performed with proper authorization. "
    "If a method could be illegal without.permission, clearly state that authorization is required."
)

for label, p in [("Without safety constraints", unsafe_like_prompt), ("With safety constraints", safer_prompt)]:
    print("=" * 80)
    print(label)
    print("=" * 80)
    print(generate(p))
    print("\n\n")

### ✅ TODO 3 – Add Safety Constraints to Your Own Prompt

Take a prompt you wrote earlier in this lab and:

1. Add explicit safety/ethics constraints.  
2. Compare outputs with and without those lines.

Note any differences in:

- Wording and disclaimers  
- Level of detail in potentially risky steps


In [ ]:
# TODO: pick one of your previous prompts and add safety constraints
original_prompt = "TODO: paste one of your previous prompts here"

safer_version = (
    original_prompt
    + "\n\nIMPORTANT: Only suggest actions that are legal, ethical, and appropriate when performed "
      "with proper authorization. If you are unsure, explain the uncertainty instead of guessing."
)

for label, p in [("Original prompt", original_prompt), ("Safer version", safer_version)]:
    print("=" * 80)
    print(label)
    print("=" * 80)
    print(generate(p))
    print("\n\n")

---
## 6. Summary — Prompting as a Security Control

In this lab you:

- Saw how **structure, roles, and formatting instructions** change outputs.  
- Practiced designing prompts for **clearer, more actionable** security guidance.  
- Experimented with prompts that explicitly request **safe, ethical** behavior.

Prompt engineering cannot replace technical controls, but it is an important **first line of defense** when working with GenAI systems in security-sensitive contexts.
